# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications_bahram.tsv

pub_date	title	venue	excerpt	authors	url_slug	paper_url
2015-07-01	Monte Carlo localization in hand-drawn maps	International Conference on Intelligent Robots and Systems (IROS)		Bahram Behzadian, Pratik Agarwal, Wolfram Burgard, and Gian Diego Tipaldi.	Paper-title-number-1	http://buiksat.github.io/files/IROS_2015.pdf
2015-10-01	Robot navigation in hand-drawn sketched maps	European conference on mobile robots (ECMR)		Federico Boniardi, Bahram Behzadian, Wolfram Burgard, and Gian Diego Tipaldi	Paper-title-number-2	http://buiksat.github.io/files/ECMR_2015.pdf
2018-01-05	Low-rank Feature Selection for Reinforcement Learning	International Symposium on Artificial Intelligence and Mathematics (ISAIM)		 Bahram Behzadian, Marek Petrik	Paper-title-number-3	http://buiksat.github.io/files/ISAIM_2018.pdf
2018-07-05	Feature Selection by Singular Value Decomposition for Reinforcement Learning	Prediction and Generative Modeling Workshop at IJCAI/ICML		 Bahram Behzadian, Marek Petrik	Paper-title-number

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications_bahram.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,authors,url_slug,paper_url
0,2015-07-01,Monte Carlo localization in hand-drawn maps,International Conference on Intelligent Robots...,NaN,"Bahram Behzadian, Pratik Agarwal, Wolfram Burg...",Paper-title-number-1,http://buiksat.github.io/files/IROS_2015.pdf
1,2015-10-01,Robot navigation in hand-drawn sketched maps,European conference on mobile robots (ECMR),NaN,"Federico Boniardi, Bahram Behzadian, Wolfram B...",Paper-title-number-2,http://buiksat.github.io/files/ECMR_2015.pdf
2,2018-01-05,Low-rank Feature Selection for Reinforcement L...,International Symposium on Artificial Intellig...,NaN,"Bahram Behzadian, Marek Petrik",Paper-title-number-3,http://buiksat.github.io/files/ISAIM_2018.pdf
3,2018-07-05,Feature Selection by Singular Value Decomposit...,Prediction and Generative Modeling Workshop at...,NaN,"Bahram Behzadian, Marek Petrik",Paper-title-number-4,http://buiksat.github.io/files/FAIM_2018.pdf
4,2019-07-05,Fast Feature Selection for Linear Value Functi...,International Conference on Automated Plannin...,NaN,"Bahram Behzadian, Soheil Gharatappeh, Marek Pe...",Paper-title-number-5,http://buiksat.github.io/files/ICAPS_2019.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\nauthors: '" + html_escape(item.authors) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
    #    md += "\n[PDF](" + item.paper_url + ")\n" 
        
    md += "\nAuthors: " + item.authors
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2015-07-01-Paper-title-number-1.md  2018-07-05-Paper-title-number-4.md
2015-10-01-Paper-title-number-2.md  2019-07-05-Paper-title-number-5.md
2018-01-05-Paper-title-number-3.md


In [22]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
